In [55]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [56]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()
    
    return documents

In [57]:
%pwd

'e:\\'

In [58]:
import os
os.chdir("../")

In [59]:
import os
print(os.getcwd())

e:\


In [60]:
%pwd

'e:\\'

In [61]:
%pwd

'e:\\'

In [62]:
extracted_data = load_pdf_file(data='E:/Medical-Chatbot-Generative-AI/Data/')

In [63]:
#extracted_data

In [65]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [67]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [69]:
#text_chunks

In [70]:
from langchain.embeddings import HuggingFaceEmbeddings

In [77]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [81]:
embeddings=download_hugging_face_embeddings()

In [82]:
query_result= embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [84]:
#query_result

In [101]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="E:/Medical-Chatbot-Generative-AI/.env")

True

In [102]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [103]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_4YJP9N_***********")

index_name = "medicalChatBot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': 'c3a1dcb9165d8388a2f5ca9370d9d8c8', 'date': 'Sat, 10 May 2025 17:23:10 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
import os
print(os.getcwd())

e:\


True